# Import Packages

In [1]:
from igor2 import binarywave as bw

import time
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import shutil

import aespm as ae

In [2]:
step = 8e-3 / 20 # 8 mm in 20 steps

folder = r"C:\Users\Asylum User\Documents\Asylum Research Data\240410"

exp = ae.Experiment(folder=folder)

# Add custom functions for this experiment

In [21]:
# Add custom functions to the experiment object

# Function to read meter
def read_meter(self, connection=None,):
    '''
    This is a read meter function.
    '''
    ae.write_spm(commands="GetMeter()", connection=self.connection)
    w = ae.ibw_read(r"C:\Users\Asylum User\Documents\buffer\Meter.ibw", connection=self.connection)
    return w.data

exp.add_func(read_meter)

# Function to set the deflection 
def set_deflection(self, defl=0.5):
    
    # Zero PD
    self.execute(action='ZeroPD', wait=3)
    
    defl_read = self.read_meter()[1]

    dart_init2 = [
        ['SetpointDefl', defl_read+defl, None],
        ['DARTTrigger', defl_read+defl, None],
    ]
    
    self.execute_sequence(operation=dart_init2)

exp.add_func(set_deflection)

# Function to move the stage with the given displacement 
def stage(self, distance):
    
    # Enable the stage move --> 5 sec, 8 seconds for safety
    self.execute('EnableStage', wait=8)
    ae.move_stage(distance=distance)
    time.sleep(2* 2)
    self.execute('DisableStage', wait=1)
    # Approaching to the sample surface --> 20 sec, let's use 30 seconds for safety
    self.execute('StartApproach', wait=45)

exp.add_func(stage)

# Function to check the file number in a given folder
def check_files(self):
    return ae.check_file_number(path=self.folder)
exp.add_func(check_files)

# Function to start and save an AC scan
def start_ac_scan(self, fname):
    ac_scan = [
            ['ChangeName', fname, None], # Change file names
            ['Capture', None, None], # Capture an optical image
            ['DownScan', None, 1.5], # Start a down scan
            ['check_files', None, None], # Check file numbers in the data save folder
        ]
    self.execute_sequence(operation=ac_scan)

exp.add_func(start_ac_scan)

# Function to start and save an AC scan
def start_dart_scan(self, fname):
    dart_scan = [
        ['ChangeName', fname, None], # Change file names
        ['DownScan', None, 1.5], # Start a down scan
        ['check_files', None, None], # Check file numbers in the data save folder
    ]
    self.execute_sequence(operation=dart_scan)

exp.add_func(start_dart_scan)

# Function to start and save an AC scan
def start_dart_ss(self, fname):
    dart_ss = [
        ['ChangeName', fname, None], # Change file names
        ['ClearForce', None, None], # Clear any existing force points
        ['GoThere', None, 1], # Move to the center of the image
        ['IVDoItDART', None, None], # Start a DART spec
        ['check_files', None, 1], # Check file numbers in the data save folder
    ]
    self.execute_sequence(operation=dart_ss)

exp.add_func(start_dart_ss)



# AC mode exploration

In [ ]:
displacement = [step, 0]
num_points = 20

# Map size
xsize, ysize = 2.5e-6, 2.5e-6

exp.execute('ScanSize', xsize)
exp.execute('Pixels', value=512)

# Initial sample height
sample_height = 1.8e-3

for i in range(num_points):
    print("Working on Location: {}/{}".format(i+1, num_points), end='\r')
   
    # Skip the first point
    if i:
        # Move the stage to the next grid point
        exe.execute('stage', value=[displacement])

    # AC scan
    exe.execute('start_ac_scan', value=['BSFO_AC{:03}_'.format(i+1)])

# DART exploration

In [8]:
# Initialize all the parameter
exp.update_param(key='f_ac', value=72.843e3)
exp.update_param(key='v_ac', value=25e-3)
exp.update_param(key='setpoint_ac', value=40e-3)

exp.update_param(key='f_dart', value=353.125e3)
exp.update_param(key='v_dart', value=1)
exp.update_param(key='f_width', value=10e3)


In [10]:
# serial actions to change from DART to AC mode 

ac_init = [
    ['AC_Mode', None, 10], # Switch mode to AC
    ['DualFreq', 0, 8],    # Turn off dual freq
    ['DriveFreq', exp.param['f_ac'], None], # Change the drive freq to f_ac
    ['DriveAmp', exp.param['v_ac'], None],  # Change the drive voltage to v_ac
    ['SetpointAmp', exp.param['setpoint_ac'], None], # Change the setpoint
]

In [22]:
# serial actions to change from DART to AC mode 

dart_init = [
    ['PFM_Mode', None, 10], # Switch mode to DART
    ['DualFreq', 1, 8],     # # Turn on dual freq
    ['DARTFreq', exp.param['f_dart'], None], # Change the drive freq to f_fart
    ['DARTSweepWidth', 100e3, None], # Change the sweep width to 100 kHz
    ['DARTAmp', exp.param['v_dart'], None], # Change the drive voltage to 1 V
    ['DARTWidth', exp.param['f_width'], None], # Change the freq separation to 10 kHz
    ['set_deflection', 0.5, None], 
]

In [ ]:
displacement = [step, 0]
tunes = []
num_points = 20

# Map size
xsize, ysize = 2.5e-6, 2.5e-6
pos0 = np.array([xsize/2, ysize/2])

exp.execute('ScanSize', xsize)
exp.execute('Pixels', value=512)

# Initial sample height
sample_height = 1.8e-3

# Make sure we start from AC mode
exp.execute_sequence(operation=dart_init)

for i in range(num_points):
    print("Working on Location: {}/{}".format(i+1, num_points), end='\r')
   
    # Skip the first point
    if i:
        # Move the stage to the next grid point
        exp.execute('stage', value=[displacement])

    # AC scan
    exp.execute('start_ac_scan', value=['BSFO_AC{:03}_'.format(i+1)])
    
    # Change to DART mode
    exp.execute_sequence(operation=dart_init)
    
    # Tune the probe
    w = tune_probe(num=2, out=True, center=350e3)
    ae.tunes.append(w)
    
    # Set the new deflection
    exp.execute(action='set_deflection', value=0.5)
    
    # DART scan
    exp.execute('start_dart_scan', value=['BSFO_DART{:03}_'.format(i+1)])

    # DART Spec
    exp.execute('start_dart_ss', value=['BSFO_SS{:03}_'.format(i+1)])

    # Change the mode back to the AC mode
    exp.execute_sequence(operation=ac_init)
    
np.savez('Tunes.npz', data=np.array(tunes))